In [62]:
!pip install gigachat tqdm

In [63]:
import re
import time
import json
import ast
import pandas as pd
from tqdm import tqdm
from gigachat import GigaChat
from gigachat.models import Chat, MessagesRole, Messages

In [64]:
df_train = pd.read_csv('/content/data/train.csv')
df_train.head(3)

,id,вопрос,варианты,категория,ответ
0,0.0,Когда был открыт закон Бойля-Мариотта?,"[""1662"", ""1762"", ""1862"", ""1962""]",-,0
1,1.0,Как найти площадь параллелограмма по векторам ...,"[""3"", ""5"", ""6"", ""0""]",аналитическая геометрия для школьников,2
2,2.0,Как реагировать на комплимент?,"[""Отрицать"", ""Смущаться и молчать"", ""Сказать «...",EQ тест,2


In [65]:
token = open('/content/secrets/key_gigachat.txt').read().strip()

# Текстовый промпт

In [80]:
def safe_parse_options(answers_str: str):
    if not isinstance(answers_str, str):
        return ["—"] * 4
    try:
        return json.loads(answers_str)
    except:
        try:
            return ast.literal_eval(answers_str)
        except:
            return ["—"] * 4

def qa_message_template(question: str, answers_str: str, category: str = "-") -> str:
    options = safe_parse_options(answers_str)
    if not options:
        options = ["—"]

    # Форматируем все варианты с их реальными индексами
    formatted_options = "\n".join([f"{i}) {opt}" for i, opt in enumerate(options)])
    cat = category if category != "-" else "не указана"


    hint = ""
    q_lower = question.lower()
    cat_lower = cat.lower()

    # Арифметика, время, дни, недели, секунды
    if "арифметика" in cat_lower or any(w in q_lower for w in ["день", "дней", "недел", "минут", "секунд", "час"]):
        hint = "\nТы — интерпретатор Python. Выполни код в уме и выбери ИНДЕКС правильного результата."

    # Программирование
    elif "программир" in cat_lower or "python" in q_lower or "print(" in question:
        hint = "Вспомни правила этого языка. ** - возведение в степень, * - умножение, + - сложение, - - вычитание, / - деление."

    # Будущее (2025, 2026) — модель не знает, просим правдоподобие
    elif "2025" in question or "2026" in question:
      hint = "\nЭто гипотетический сценарий. Известные/действующие персоналии и события — вероятно, НЕ отменены и НЕ избраны."

    # Логика, наука, определения
    elif any(kw in cat_lower for kw in ["антрополог", "геометри", "шахмат", "физик", "биолог", "истор", "оптика", "математик"]):
        hint = "\nВыбери наиболее точное определение или факт."

    prompt = f"""Ты — эксперт по вопросам с множественным выбором.
Вопрос: {question}
Варианты:
{formatted_options}
Категория: {cat}{hint}

Правила:
- Ответ должен быть ТОЛЬКО индексом правильного варианта (целое число, больше или равно 0).

Ответ:"""
    return prompt

# Парсинг ответа

In [67]:
def parse_prediction(raw_output: str) -> int:
    digits = re.findall(r'\d+', raw_output.strip())
    if digits:
        return int(digits[0])
    return 0

# Тестовый пример для проверки корректности промпта

In [68]:
def test_run_model(idx: int):
  row = df_train.iloc[idx]

  print("=== Prompt ===")
  question = row['вопрос']
  answers_str = row['варианты']
  category = row['категория']
  true_ans = int(row['ответ'])

  prompt = qa_message_template(question, answers_str, category)
  print(prompt)

  print("\n=== Answer Model ===")
  with GigaChat(credentials=token, verify_ssl_certs=False) as giga:
    payload = {
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 3
        }
    resp = giga.chat(payload)
    raw = resp.choices[0].message.content
    print(repr(raw))

  pred = parse_prediction(raw)

  print(f"\nПравильный: {true_ans}, Предсказано: {pred}, Верно: {pred == true_ans}")

In [69]:
test_run_model(29)

=== Prompt ===
Ты — эксперт по вопросам с множественным выбором.
Вопрос: Что выведет программа: `print(2 + 3 * 4)`?
Варианты:
0) 20
1) 14
2) 24
3) 10
Категория: задачи по программированию для школьников
Ты — интерпретатор Python. Помни: ** — возведение в степень, * и / перед + и -, числа с . — float. Выведи ИНДЕКС.

Правила:
- Ответ должен быть ТОЛЬКО индексом правильного варианта (целое число, больше или равно 0).

Ответ:

=== Answer Model ===
'3'

Правильный: 1, Предсказано: 3, Верно: False


# Предсказывание модели

In [101]:
def predict_with_gigachat(
    df,
    token,
    model,
    max_tokens,
    max_retries=3,
    delay_between_requests=0.5,
    timeout=60,
    batch_size=50,
    delay_between_batches=1.5
):
    results = []
    with GigaChat(credentials=token, verify_ssl_certs=False, timeout=timeout) as giga:
        for i, (_, row) in enumerate(tqdm(df.iterrows(), total=len(df), desc="GigaChat QA")):
            record_id = row['id']
            question = row['вопрос']
            answers = row['варианты']
            category = row.get('категория', '-')

            prediction = 1

            for attempt in range(max_retries):
                try:
                    # Формируем промпт с категорией
                    prompt = qa_message_template(question, answers, category)

                    # Отправляем запрос с ограничением длины ответа
                    payload = {
                      "model": model,
                      "messages": [{"role": "user", "content": prompt}],
                      "max_tokens": max_tokens,
                      "temperature": 0.1,
                      "top_p": 0.9
                    }
                    response = giga.chat(payload)

                    # Безопасное извлечение контента
                    raw_output = response.choices[0].message.content
                    if raw_output is None:
                        raw_output = ""

                    prediction = parse_prediction(raw_output)
                    break  # успех — выходим

                except Exception as e:
                    print(f"\nОшибка id={record_id}, попытка {attempt + 1}/{max_retries}: {e}")
                    time.sleep(1 + attempt * 2)

            results.append({'id': record_id, 'prediction': prediction})

            # Задержки
            if delay_between_requests > 0:
                time.sleep(delay_between_requests)
            if (i + 1) % batch_size == 0 and (i + 1) < len(df):
                time.sleep(delay_between_batches)

    return pd.DataFrame(results)

In [85]:
model = "GigaChat-Max"
max_tokens = 5

# Для train (проверка accuracy)
df_pred = predict_with_gigachat(df_train, model=model, max_tokens=max_tokens, token=token)
df_merged = pd.merge(df_train[['id', 'ответ']], df_pred, on='id')
accuracy = (df_merged['ответ'] == df_merged['prediction']).mean()
print(f"Train accuracy: {accuracy:.2%}")

GigaChat QA: 100%|██████████| 87/87 [00:27<00:00,  3.21it/s]

Train accuracy: 95.40%


# Смотрим ошибки

In [86]:
errors = df_train.merge(df_pred, on='id')
errors = errors[errors['ответ'] != errors['prediction']]

for _, row in errors.head(20).iterrows():
    print(f"\nID: {row['id']}")
    print(f"Категория: {row['категория']}")
    print(f"Вопрос: {row['вопрос']}")
    print(f"Варианты: {row['варианты']}")
    print(f"Правильный: {row['ответ']}, Предсказано: {row['prediction']}")


ID: 11.0
Категория: -
Вопрос: Какой международный спортивный турнир был отменён в 2025 году из-за климатических условий?
Варианты: ["Чемпионат мира по футболу", "Летние Олимпийские игры", "Australian Open", "Тур де Франс", "Кубок Америки", "Чемпионат мира по лёгкой атлетике", "Нет отменённых турниров"]
Правильный: 2, Предсказано: 6

ID: 16.0
Категория: задачи по программированию для школьников
Вопрос: Что выведет код: `print(2 ** 3 + 1)`?
Варианты: ["6", "7", "8", "9", "10", "5", "12", "16", "4", "3"]
Правильный: 3, Предсказано: 1

ID: 35.0
Категория: Transhumanism Inc 2026
Вопрос: В какую эпоху погружается Маркус в симуляции в ромате A Sinstar Пелевина?
Варианты: ["XVII век", "XVI век", "XV век", "XVIII век"]
Правильный: 1, Предсказано: 0

ID: 61.0
Категория: новости 2025
Вопрос: Кто был избран генеральным секретарём ООН в 2025 году?
Варианты: ["Антониу Гутерриш", "Кристалина Георгиева", "Мария Фернанда Эспиноза", "Саманта Пауэр", "Хелен Кларк", "Нгоци Оконджо-Ивеала", "Джон Керри"]


# Загрузка тестовых данных

In [90]:
df_test = pd.read_csv('/content/data/test.csv')
df_test.head(3)

,id,вопрос,варианты,категория
0,0.0,Какая страна станет хозяйкой Кубка африканских...,"[""Египет"", ""Марокко"", ""ЮАР"", ""Нигерия""]",-
1,1.0,Какой процесс привёл к появлению торговли?,"[""Неолитическая революция"", ""Промышленная рево...",антропология
2,2.0,"Как называется физическая величина, равная отн...","[""Разрешающая способность"", ""Увеличение"", ""Фок...",оптика


In [91]:
len(df_test)

786

In [102]:
model = "GigaChat"
max_tokens = 5
df_results = predict_with_gigachat(df_test, model=model, max_tokens=max_tokens, token=token)


GigaChat QA:  23%|██▎       | 180/786 [01:57<06:22,  1.58it/s]<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
GigaChat QA:  27%|██▋       | 215/786 [02:21<05:59,  1.59it/s]<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
GigaChat QA:  62%|██████▏   | 488/786 [05:20<03:08,  1.58it/s]


Ошибка id=488.0, попытка 1/3: 'float' object has no attribute 'lower'

Ошибка id=488.0, попытка 2/3: 'float' object has no attribute 'lower'

Ошибка id=488.0, попытка 3/3: 'float' object has no attribute 'lower'


GigaChat QA:  80%|████████  | 629/786 [07:02<01:38,  1.60it/s]


Ошибка id=629.0, попытка 1/3: 'float' object has no attribute 'lower'

Ошибка id=629.0, попытка 2/3: 'float' object has no attribute 'lower'

Ошибка id=629.0, попытка 3/3: 'float' object has no attribute 'lower'


GigaChat QA:  81%|████████▏ | 639/786 [07:17<01:47,  1.37it/s]


Ошибка id=639.0, попытка 1/3: 'float' object has no attribute 'lower'

Ошибка id=639.0, попытка 2/3: 'float' object has no attribute 'lower'

Ошибка id=639.0, попытка 3/3: 'float' object has no attribute 'lower'


GigaChat QA:  90%|█████████ | 708/786 [08:12<00:51,  1.52it/s]<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
<unknown>:1: SyntaxWarning: invalid escape sequence '\p'
GigaChat QA: 100%|██████████| 786/786 [09:03<00:00,  1.45it/s]


In [103]:
df_results.to_csv('submission.csv', index = False)